In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

## Importar dataset

In [2]:
import tensorflow_datasets as tfds

In [3]:
dataset = tfds.load('imdb_reviews', as_supervised=True)

In [4]:
raw_train_ds, raw_test_ds = dataset['train'], dataset['test']

In [5]:
for text, label in raw_train_ds.take(1):
    print(text.numpy(), label.numpy())

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it." 0


## Preparar dataset

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = tf.data.experimental.cardinality(raw_train_ds)
BUFFER_SIZE.numpy()

25000

In [7]:
batch_size = 128
voc_size = 5000

train_ds = raw_train_ds.shuffle(BUFFER_SIZE).batch(
        batch_size, num_parallel_calls=AUTOTUNE).prefetch(
        AUTOTUNE)

val_ds = raw_test_ds.batch(
        batch_size, num_parallel_calls=AUTOTUNE).prefetch(
        AUTOTUNE)

In [8]:
for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(128,), dtype=string, numpy=
array([b'Greetings again from the darkness. Remember all the "What happened to Woody Allen?" jokes? Even Mr. Allen poked fun at the fans who wanted him to continue making his same "funny" films. As with any great artist, Mr. Allen\'s craft evolved over the years and he lost some fans, while picking up others. Last year\'s masterpiece "Matchpoint" showed he is still every bit as relevant and poignant as he was in the days of "Annie Hall" and "Manhattan". What is most striking to us 40 plus year fans is that Mr. New York himself seems to have a bit of a crush on the mother country. Apparently he actually likes England!! While filming "Matchpoint", Mr. Allen became enamored with Scarlett Johansson and her real life spirit and sense of humor. This attraction motivated him to write his best comedy in years. Scarlett, while risking overexposure, must be given credit for not just picking films that cast her in some glamorous light. She is unafra

## Tokenización

In [9]:
seq_length = 128
vectorize_layer = layers.TextVectorization(
    max_tokens=voc_size,
    output_mode='int',
    output_sequence_length=seq_length)

- Adaptar la capa

In [10]:
vectorize_layer_ds = train_ds.map(lambda text, label: text)
vectorize_layer.adapt(vectorize_layer_ds)

In [11]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'br',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but',
 'film',
 'on',
 'not',
 'you',
 'are',
 'his',
 'have',
 'he',
 'be',
 'one',
 'its',
 'at',
 'all',
 'by',
 'an',
 'they',
 'from',
 'who',
 'so',
 'like',
 'her',
 'just',
 'or',
 'about',
 'has',
 'if',
 'out',
 'some',
 'there',
 'what',
 'good',
 'when',
 'more',
 'very',
 'even',
 'she',
 'my',
 'no',
 'up',
 'would',
 'which',
 'only',
 'time',
 'really',
 'story',
 'their',
 'were',
 'had',
 'see',
 'can',
 'me',
 'than',
 'we',
 'much',
 'well',
 'been',
 'get',
 'will',
 'into',
 'also',
 'because',
 'other',
 'do',
 'people',
 'bad',
 'great',
 'first',
 'how',
 'most',
 'him',
 'dont',
 'made',
 'then',
 'movies',
 'make',
 'films',
 'could',
 'way',
 'them',
 'any',
 'too',
 'after',
 'characters',
 'think',
 'watch',
 'two',
 'many',
 'being',
 'seen',
 'character',
 'never',
 'little',
 'acting',
 'where',
 'plot',
 'be

- Probar vectorize_layer con batch de prueba

In [12]:
test_batch = tf.constant([['Hi there']])
vectorize_layer(test_batch)

<tf.Tensor: shape=(1, 128), dtype=int64, numpy=
array([[ 1, 48,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>

## Definir LSTM

In [13]:
lstm = tf.keras.Sequential([
    vectorize_layer,
    layers.Embedding(
        input_dim=voc_size, output_dim=128),
    layers.Bidirectional(layers.LSTM(128)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)

])

- Probar LSTM con batch de prueba

In [14]:
lstm(test_batch)

Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.02419756]], dtype=float32)>

- Información del modelo

In [15]:
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 128)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 128, 128)          640000    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 919,681
Trainable params: 919,681
Non-trai

## Entrenamiento LSTM

In [16]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [17]:
lstm_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [18]:
train_loss_avg = tf.keras.metrics.Mean(name='train_loss')
val_loss_avg = tf.keras.metrics.Mean(name='train_loss')

In [19]:
epochs = 5

In [20]:
@tf.function
def train_step(text, target):
    with tf.GradientTape() as tape:
        logits = lstm(text, training=True)
        loss_value = loss(tf.cast(target, tf.float32), logits)

    gradients = tape.gradient(loss_value, lstm.trainable_weights)
    lstm_opt.apply_gradients(zip(gradients, lstm.trainable_weights))
    train_loss_avg(loss_value)
    
@tf.function
def val_step(text, target):
    with tf.GradientTape() as tape:
        logits = lstm(text, training=False)
        loss_value = loss(tf.cast(target, tf.float32), logits)

    val_loss_avg(loss_value)

In [21]:
for epoch in range(epochs):
    for text, target in train_ds:
        train_step(text, target)
        
    print(f'Epoch: {epoch} Train loss: {train_loss_avg.result().numpy()}')
    train_loss_avg.reset_states()
    
    for text, target in val_ds:
        val_step(text, target)
        
    print(f'Val loss: {val_loss_avg.result().numpy()}')
    val_loss_avg.reset_states()

Epoch: 0 Train loss: 0.6552780866622925
Val loss: 0.5156950950622559
Epoch: 1 Train loss: 0.40275484323501587
Val loss: 0.38369372487068176
Epoch: 2 Train loss: 0.3245213031768799
Val loss: 0.39837175607681274
Epoch: 3 Train loss: 0.29422077536582947
Val loss: 0.37711358070373535
Epoch: 4 Train loss: 0.2717871367931366
Val loss: 0.39608845114707947


## Definir Transformer

In [22]:
def scaled_dot_product(q, k, v):
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_qk = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(dk)
    attn_weights = tf.nn.softmax(scaled_qk, axis=-1)  
    output = tf.matmul(attn_weights, v) 
    return output

In [23]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, model_dim, n_heads):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.model_dim = model_dim

        assert model_dim % self.n_heads == 0

        self.depth = model_dim // self.n_heads

        self.wq = layers.Dense(model_dim)
        self.wk = layers.Dense(model_dim)
        self.wv = layers.Dense(model_dim)

        self.dense = layers.Dense(model_dim)

    def split_into_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.n_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  
        k = self.wk(k)  
        v = self.wv(v)  

        q = self.split_into_heads(q, batch_size)  
        k = self.split_into_heads(k, batch_size)  
        v = self.split_into_heads(v, batch_size)  

        scaled_attention = scaled_dot_product(q, k, v)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) 
        original_size_attention = tf.reshape(scaled_attention, (batch_size, -1, self.model_dim)) 

        output = self.dense(original_size_attention) 
        return output

In [24]:
class TokenEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emb_dim, 
                 rate=0.1):
        super(TokenEmbedding, self).__init__()
        self.max_len = maxlen
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=emb_dim)
        self.position_emb = layers.Embedding(
            input_dim=maxlen, output_dim=emb_dim)
        self.dropout = layers.Dropout(rate)

    def call(self, x):
        token_embeddings = self.token_emb(x)
        positions = tf.range(start=0, limit=self.max_len, delta=1)
        positions = self.position_emb(positions)
        return self.dropout(token_embeddings + positions) 
    
pos_emb = TokenEmbedding(128, voc_size, 256)   
pos_emb(tf.ones([1, 128]))

<tf.Tensor: shape=(1, 128, 256), dtype=float32, numpy=
array([[[-0.01184506,  0.01336251,  0.01872568, ..., -0.07880308,
         -0.0362705 , -0.04052626],
        [ 0.06096837,  0.03114442,  0.02782729, ..., -0.04698677,
         -0.03798749,  0.0098971 ],
        [ 0.03586776,  0.05373102,  0.0847071 , ..., -0.06922714,
         -0.02847015, -0.06074126],
        ...,
        [-0.03307806,  0.07455195,  0.0324003 , ..., -0.08637513,
          0.01148258, -0.06306032],
        [ 0.00860722,  0.00564646,  0.05508369, ..., -0.04480239,
         -0.01183645, -0.0351478 ],
        [ 0.06202748,  0.05346694,  0.08869489, ..., -0.06548816,
          0.01126273, -0.05356576]]], dtype=float32)>

In [25]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, model_dim, n_heads=2, mlp_dim=512, 
                 rate=0.0, eps=1e-6):
        super(TransformerBlock, self).__init__()
        self.attn = MultiHeadAttention(model_dim, n_heads)
        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation='gelu'), 
            layers.Dense(model_dim),
        ])
        self.norm1 = layers.LayerNormalization(epsilon=eps)
        self.norm2 = layers.LayerNormalization(epsilon=eps)
        self.drop1 = layers.Dropout(rate)
        self.drop2 = layers.Dropout(rate)

    def call(self, inputs, training):  
        attn_output = self.attn(inputs, inputs, inputs)
        attn_output = self.drop1(attn_output, training=training) 
        x_norm1 = self.norm1(attn_output + inputs)
        
        mlp_output = self.mlp(x_norm1)
        mlp_output = self.drop2(mlp_output, training=training)
        return self.norm2(mlp_output + x_norm1)
    
block = TransformerBlock(128)

In [26]:
class Transformer(tf.keras.models.Model):
    def __init__(self, model_dim, voc_size, mlp_dim=256, 
                 seq_length=128, heads=4):
        super(Transformer, self).__init__()
        self.emb = TokenEmbedding(seq_length, voc_size, model_dim)
        self.block = TransformerBlock(model_dim, heads, mlp_dim)
        self.out = tf.keras.Sequential([
            layers.GlobalAveragePooling1D(),
            layers.Dense(1)
        ])
    
    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        x = self.block(x)
        x = self.out(x)
        return x
    
transformer = Transformer(128, voc_size)
transformer(test_batch)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7201242]], dtype=float32)>

In [27]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 token_embedding_1 (TokenEmb  multiple                 656384    
 edding)                                                         
                                                                 
 transformer_block_1 (Transf  multiple                 132480    
 ormerBlock)                                                     
                                                                 
 sequential_3 (Sequential)   (1, 1)                    129       
                                                                 
Total params: 788,993
Trainable params: 788,993
Non-trainable params: 0
_________________________________________________________________


## Entrenamiento Transformer
- Utilizar los mismos parámteros de lstm

In [28]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [29]:
trans_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [30]:
train_loss_avg = tf.keras.metrics.Mean(name='train_loss')
val_loss_avg = tf.keras.metrics.Mean(name='train_loss')

In [31]:
epochs = 5

In [32]:
@tf.function
def train_step(text, target):
    with tf.GradientTape() as tape:
        logits = transformer(text, training=True)
        loss_value = loss(tf.cast(target, tf.float32), logits)

    gradients = tape.gradient(loss_value, transformer.trainable_weights)
    trans_opt.apply_gradients(zip(gradients, transformer.trainable_weights))
    train_loss_avg(loss_value)
    
@tf.function
def val_step(text, target):
    with tf.GradientTape() as tape:
        logits = transformer(text, training=False)
        loss_value = loss(tf.cast(target, tf.float32), logits)

    val_loss_avg(loss_value)

In [33]:
for epoch in range(epochs):
    for text, target in train_ds:
        train_step(text, target)
        
    print(f'Epoch: {epoch} Train loss: {train_loss_avg.result().numpy()}')
    train_loss_avg.reset_states()
    
    for text, target in val_ds:
        val_step(text, target)
        
    print(f'Val loss: {val_loss_avg.result().numpy()}')
    val_loss_avg.reset_states()

Epoch: 0 Train loss: 0.5846831798553467
Val loss: 0.43435412645339966
Epoch: 1 Train loss: 0.37525197863578796
Val loss: 0.39283284544944763
Epoch: 2 Train loss: 0.32010599970817566
Val loss: 0.38050511479377747
Epoch: 3 Train loss: 0.2931949198246002
Val loss: 0.38015273213386536
Epoch: 4 Train loss: 0.27938756346702576
Val loss: 0.42643195390701294


## Ejercicio

- Modificar hiperparámetros de los modelos para obtener mejores resultados.
- Modificar las arquitecturas.
- Agregar y modificar regularización.